In [2]:
import requests
import numpy as np
import pandas as pd
import os
import traceback

In [67]:
def get_contents(baseurl, content, start_rev_id, end_rev_id=""):
    content_url = os.path.join(baseurl, "rev_content", content, str(start_rev_id)+"/")
    if end_rev_id:
        content_url = os.path.join(content_url, str(end_rev_id)+"/")
    params = { "o_rev_id": "false", "editor": "false", "token_id": "true", "in": "false", "out": "false" }
    try:
        response = requests.get(content_url, params= params)
        if response.status_code == requests.codes.ok: 
            response = response.json()
            if "revisions" in response.keys() :
                return response["revisions"]
            elif "revisions" not in response.keys() : 
                raise AttributeError("Server did not return revisions key it returned \t"+response.keys())
        elif response.status_code != requests.codes.ok : 
            print(content_url)
            raise AttributeError("Server returned bad code\t"+response.status_code)
    except:
        print(traceback.format_exc())

In [4]:
def tokens_to_df(tokens):
    tokens.insert(0, {'token_id':-1, 'str':  "{st@rt}"})
    tokens.append({'token_id':-2, 'str': "{$nd}"})
    return pd.Series(tokens)

In [17]:
def save_content(revison_series, filename, content, step=200, baseurl="https://api.wikiwho.net/en/api/v1.0.0-beta/"):
    end_index = revison_series.size
    from_index = 0
    with pd.HDFStore(filename, 'a') as store:
        try:
            for to_index in  range(0, end_index, step):    
                rev_contents = get_contents(baseurl, content, str(revison_series[from_index]), str(revison_series[to_index]))
                from_index = to_index
                for rev_content in rev_contents:
                    key = "r"+list(rev_content.keys())[0]
                    df = tokens_to_df(list(rev_content.values())[0]["tokens"])
                    store.put(key, df, table=False)
            to_index = from_index + (end_index-1)%step
            rev_contents = get_contents(baseurl, content, str(revison_series[from_index]), str(revison_series[to_index]))
            rev_contents.extend(get_contents(baseurl, content, str(revison_series[to_index])))
            for rev_content in rev_contents:
                key = "r"+list(rev_content.keys())[0]
                df = tokens_to_df(list(rev_content.values())[0]["tokens"])
                store.put(key, df, table=False)
        except:
            print("problem ", traceback.format_exc())

In [93]:
def save_article(article_name, baseurl="https://api.wikiwho.net/en/api/v1.0.0-beta/", save_dir = "../data/content", step=200):
    params = {"editor": "true", "timestamp": "true"}
    filename = article_name + ".h5"
    revisions_url = os.path.join( baseurl, "rev_ids", article_name+"/")
    response = requests.get(revisions_url, params= params)
    revisons_list = response.json()["revisions"]
    rev_list_df = pd.DataFrame(revisons_list)
    save_path = os.path.join(save_dir, filename)
    with pd.HDFStore(save_path, 'a') as store:
        store.put("rev_list", rev_list_df, table=False)
    save_content(rev_list_df["id"], save_path, article_name, step=step)

In [88]:
def tokens_to_list(tokens):
    token_ids =   [ token["token_id"] for token in tokens ]

    token_ids.insert(0, -1)
    token_ids.append(-2)
    return pd.DataFrame(token_ids)

In [89]:
def save_content_list(revison_series, filename, content, step=200, baseurl="https://api.wikiwho.net/en/api/v1.0.0-beta/"):
    end_index = revison_series.size
    from_index = 0
    with pd.HDFStore(filename, 'a') as store:
        try:
            for to_index in  range(0, end_index, step):    
                rev_contents = get_contents(baseurl, content, str(revison_series[from_index]), str(revison_series[to_index]))
                from_index = to_index
                for rev_content in rev_contents:
                    key = "r"+list(rev_content.keys())[0]
                    df = tokens_to_list(list(rev_content.values())[0]["tokens"])
                    store.put(key, df, table=False)
            # 
            to_index = from_index + (end_index-1)%step
            rev_contents = get_contents(baseurl, content, str(revison_series[from_index]), str(revison_series[to_index]))
            rev_contents.extend(get_contents(baseurl, content, str(revison_series[to_index])))
            for rev_content in rev_contents:
                key = "r"+list(rev_content.keys())[0]
                df = tokens_to_list(list(rev_content.values())[0]["tokens"])
                store.put(key, df, table=False)
        except:
            print("problem ", traceback.format_exc())

In [105]:
%%time
baseurl = "https://api.wikiwho.net/en/api/v1.0.0-beta/"
content = "Cloning"
save_dir = "../data/content"
params = {"editor": "true", "timestamp": "true"}
filename = content + "1.h5"
save_path = os.path.join(save_dir, filename)
revisions_url = os.path.join( baseurl, "rev_ids", content+"/")
response = requests.get(revisions_url, params= params)
revisons_list = response.json()["revisions"]
rev_list_df = pd.DataFrame(revisons_list)
# print(save_path)
# save_content_list(rev_list_df["id"], save_path, content, step=200)

CPU times: user 84 ms, sys: 8 ms, total: 92 ms
Wall time: 3.56 s


In [76]:
%%time
rev_contents = {}
# for revision in revisons_list[50:160]:
#     rev_id = str(revision["id"])
#     response = get_contents(baseurl, content, rev_id)
#     rev_contents[revision["id"]] = [ token["token_id"] for token in response[0][ rev_id ]["tokens"] ]
# with pd.HDFStore(filename, 'a') as store:
#     store.put("rev_list", rev_list_df, table=False)

# %time save_content(rev_list_df["id"], filename, content, step=200)

# %time save_content(rev_list_df["id"], filename, content, step=50)

# %time save_content(rev_list_df["id"], filename, content, step=20)

https://api.wikiwho.net/en/api/v1.0.0-beta/rev_content/Violence_against_Muslims_in_India/561890618/
Traceback (most recent call last):
  File "<ipython-input-67-cf37cfe74b70>", line 16, in get_contents
    raise AttributeError("Server returned bad code\t"+response.status_code)
TypeError: must be str, not int



TypeError: 'NoneType' object is not subscriptable

In [80]:
article_series=pd.read_csv("../conflicted_article.csv")["articles"]

In [94]:
save_article(article_series[7])

In [ ]:
%%time
for article in article_series[1:]:
    print(article)
    save_article(article)

Christianity


In [28]:
article_name = article_series[]
file_name = article_name + ".h5"
save_path = os.path.join("../data/content", file_name)

In [30]:
article_series

0                   Friedrich_Nietzsche
1                          Christianity
2                              HIV/AIDS
3                               Cloning
4                              Gambling
5                              Crusades
6                       White_Americans
7                            Yugoslavia
8                                 Tibet
9                   Macedonian_language
10    Violence_against_Muslims_in_India
11         IQ_and_the_Wealth_of_Nations
12         Systemic_lupus_erythematosus
13                        Human_cloning
14                          Solar_power
15                    Electronic_voting
16                          Berlin_Wall
17                           Censorship
18                                Truth
19                         Panama_Canal
Name: articles, dtype: object